# Yahoo! Finance market data downloader

- [Intro](https://towardsdatascience.com/a-comprehensive-guide-to-downloading-stock-prices-in-python-2cd93ff821d4)


- [yfinance](https://github.com/ranaroussi/yfinance)

Very easy to use

- [yahoofinancials](https://github.com/JECSand/yahoofinancials)

A python module that returns stock, cryptocurrency, forex, mutual fund, commodity futures, ETF, and US Treasury financial data from Yahoo Finance.

The package depends on beautifulsoup4 and pytz to work


## this notebook downloads historical price data from yahoo finance


```
tsla_df = yf.download('TSLA', 
                      start='2019-01-01', 
                      end='2019-12-31', 
                      progress=False)
tsla_df.head()
```

By default, the function downloads daily data, but we can specify the interval as one of the following: 1m, 5m, 15m, 30m, 60m, 1h, 1d, 1wk, 1mo, and more


We can download the stock prices of multiple assets at once, by providing a list (such as [‘TSLA', ‘FB', ‘MSFT']) as the tickers argument. Additionally, we can set auto_adjust = True , so all the presented prices are adjusted for potential corporate actions, such as splits.




adapted from 
- ~/projects/py4kids/lesson-14-db/PostgreSQL/lesson-14-postgresql-load-csv.ipynb
- ~/projects/algorithmic-trading-python/mom101/yfinance.ipynb

In [75]:
import pandas as pd
import datetime as dt
import yfinance as yf 

In [57]:
from sqlalchemy import create_engine
import db_params

In [77]:
from time import sleep

In [58]:
db_kwargs = db_params.get_db_params(
        db_name="gwgdb", 
        db_type="postgresql"
    )

In [59]:
engine = create_engine(f'postgresql://{db_kwargs["user"]}:{db_kwargs["password"]}@{db_kwargs["host"]}:{db_kwargs["port"]}/{db_kwargs["database"]}')

In [61]:
column_types = {
    "date": "character varying", 
    "symbol": "character varying", 
    "open": "numeric",
    "high": "numeric",
    "low": "numeric",
    "close": "numeric",
   # "adj_close": "numeric",
    "volume": "integer",    
    "chg_2day": "numeric",
    "chg_5day": "numeric",
    "chg_20day": "numeric",
    "chg_50day": "numeric",
    "chg_100day": "numeric",
    "chg_200day": "numeric",
    "ma15": "numeric",
    "ma50": "numeric",
    "ma200": "numeric",
    "slp_ma15": "numeric",
    "slp_ma50": "numeric",
    "slp_ma200": "numeric",
    "chg_2day": "numeric",
    "chg_5day": "numeric",
    "chg_20day": "numeric",
    "chg_50day": "numeric",
    "chg_100day": "numeric",
    "chg_200day": "numeric",
    "vol_ma20": "integer", 
    "comments": "text"
}

In [62]:
table_name = "quote_daily"
owner = "gwguser001"
ddl_str = f"""
DROP TABLE public.{table_name};

CREATE TABLE public.{table_name} (
"""
ncols = len(column_types.keys())
i = 0
for k,v in column_types.items():
    i += 1
    ddl_str += "\t"
    if k in ["date", "symbol"]:
        ddl_str += f"{k}  {v} NOT NULL,"
    else:
        ddl_str += f"{k}  {v},"
    ddl_str += "\n"

ddl_str += f"\tCONSTRAINT {table_name}_pkey PRIMARY KEY (date, symbol)\n"
ddl_str += ");\n\n"

ddl_str += f"ALTER TABLE public.{table_name} OWNER to {owner};\n"

In [63]:
print(ddl_str)


DROP TABLE public.quote_daily;

CREATE TABLE public.quote_daily (
	date  character varying NOT NULL,
	symbol  character varying NOT NULL,
	open  numeric,
	high  numeric,
	low  numeric,
	close  numeric,
	volume  integer,
	chg_2day  numeric,
	chg_5day  numeric,
	chg_20day  numeric,
	chg_50day  numeric,
	chg_100day  numeric,
	chg_200day  numeric,
	ma15  numeric,
	ma50  numeric,
	ma200  numeric,
	slp_ma15  numeric,
	slp_ma50  numeric,
	slp_ma200  numeric,
	vol_ma20  integer,
	comments  text,
	CONSTRAINT quote_daily_pkey PRIMARY KEY (date, symbol)
);

ALTER TABLE public.quote_daily OWNER to gwguser001;



In [18]:
!ls unique*.csv

unique-tickers.csv


In [21]:
tickers = pd.read_csv("unique-tickers.csv")["Ticker"].tolist()

In [22]:
len(tickers), tickers[:2]

(5142, ['A', 'AA'])

In [37]:
NUM_DAYS = 760

In [78]:
print(f"start time: {dt.datetime.now()}")

chunk_size = 100
for i in range(0, len(tickers), chunk_size):
    print(f"i={i}")

    for j in range(chunk_size):
        ij = i+j
        if ij >= len(tickers): 
            break

        symbol = tickers[ij]
        if symbol in ['A'] or symbol <= "AEHR": 
            continue
        
        print(symbol)

        df = yf.Ticker(symbol).history(f"{NUM_DAYS}d")

        # df.shape
        df["date"] = df.index.strftime("%Y-%m-%d")
        df["symbol"] = symbol
        df.rename(columns={'Open': 'open', 'High': 'high',
                           'Low': 'low', 'Close': 'close', "Volume": "volume"
                          }, inplace=True)
        # df.columns
        df = df[['date', 'symbol', 'open', 'high', 'low', 'close', 'volume']]
        df.reset_index(drop=True, inplace=True)

        # https://datatofish.com/pandas-dataframe-to-sql/
        df.to_sql('quote_daily', engine, if_exists='append', index=False)
        
    print(f"end time: {dt.datetime.now()}")    
    
    sleep(3)  # sleep 3 sec

start time: 2020-12-26 00:19:58.630010
i=0
end time: 2020-12-26 00:19:58.630553
i=100
AEIS
AEL
AEM
AEMD
AENZ
AEO
AEP
AER
AERI
AES
AESE
AEY
AEYE
AEZS
AFG
AFI
AFIB
AFIN
AFL
AFMD
AFYA
AG
AGBA
AGCO
AGE
AGEN
AGFS
AGI
AGIO
AGLE
AGM
AGMH
AGO
AGR
AGRO
AGRX
AGS
AGTC
AGX
AGYS
AHC
AHCO
AHH
AHPI
AI
AIG
AIH
AIHS
AIKI
AIM
AIMC
AIN
AINC
AINV
AIR
AIRG
AIRI
AIRT
AIT
AIV
AIZ
AJG
AJRD
AJX
AKAM
AKBA
AKER
AKR
AKRO
AKTS
AKTX
AKU
AKUS
AL
ALAC
ALB
ALBO
ALC
ALCO
ALDX
ALE
ALEC
ALEX
ALG
ALGM
ALGN
ALGS
ALGT
ALIM
ALJJ
ALK
ALKS
ALL
ALLE
ALLK
ALLO
ALLT
ALLY
ALNA
ALNY
end time: 2020-12-26 00:20:41.969020
i=200
ALOT
ALPN
ALRM
ALRN
ALRS
ALSK
ALSN
ALT
ALTA
ALTG
ALTM
ALTR
ALUS
ALV
ALVR
ALX
ALXN
ALXO
ALYA
AM
AMAL
AMAT
AMBA
AMBC
AMBO
AMC
AMCI
AMCR
AMCX
AMD
AME
AMED
AMEH
AMG
AMGN
AMHC
AMK
AMKR
AMN
AMNB
AMOT
AMOV
AMP
AMPE
AMPH
AMPY
AMRB
AMRC
AMRH
AMRK
AMRN
AMRS
AMRX
AMS
AMSC
AMSF
AMST
AMSWA
AMT
AMTB
AMTBB
AMTI
AMTX
AMWD
AMWL
AMX
AMYT
AMZN
AN
ANAB
ANAT
ANCN
ANDA
ANDE
ANET
ANF
ANGI
ANGO
ANH
ANIK
ANIP
ANIX
ANNX
ANPC
ANSS
ANTE
A

FDBC
FDP
FDS
FDUS
FDX
FE
FEAC
FEDU
FEIM
FELE
FENC
FENG
FET
FEYE
FF
FFBC
FFBW
FFG
FFHL
FFIC
FFIN
FFIV
FFNW
FFWM
FGBI
FGEN
FGNA
FHB
FHI
FHN
FHTX
FI
FIBK
FICO
FIII
FINV
FIS
FISI
FISV
FIT
FITB
FIVE
FIVN
FIX
FIXX
FIZZ
FL
FLDM
FLEX
FLGT
FLIC
FLIR
FLL
FLMN
FLNG
FLNT
FLO
FLOW
FLR
FLS
FLT
FLUX
FLWS
FLXN
FLXS
FLY
FMAC
FMAO
FMBH
FMBI
FMC
FMNB
FMS
FMTX
FMX
FN
FNB
FNCB
FND
FNF
FNHC
FNKO
FNLC
FNV
FNWB
FOCS
FOE
end time: 2020-12-26 00:30:52.169615
i=1800
FOLD
FONR
FOR
FORD
FORM
FORR
FORTY
FOSL
FOUR
FOX
FOXA
FOXF
FPAY
FPI
FPRX
FRAF
FRAN
FRBA
FRBK
FRC
FRD
FREE
FREQ
FRG
FRGI
FRHC
FRLN
FRME
FRO
FROG
FRPH
FRPT
FRSX
FRT
FRTA
FSB
FSBW
FSDC
FSEA
FSFG
FSI
FSK
FSKR
FSLF
FSLR
FSLY
FSM
FSP
FSRV
FSS
FSTR
FSTX
FT
FTAI
FTCH
FTDR
FTEK
FTFT
FTHM
FTI
FTIV
FTK
FTNT
FTOC
FTS
FTSI
FTV
FUBO
FUL
FULC
FULT
FUN
FUNC
FUND
FUSB
FUSE
FUSN
FUTU
FUV
FVAM
FVCB
FVE
FVRR
FWONA
FWONK
FWP
FWRD
FXNC
G
GABC
GAIA
GAIN
GALT
GAM
GAN
GASS
GATO
GATX
GAU
GB
end time: 2020-12-26 00:31:22.470932
i=1900
GBAB
GBCI
GBDC
GBIO
GBL
GBLI
GBR
GBT
GBX
G

OCCI
OCFC
OCFT
OCGN
OCN
OCSI
OCSL
OCUL
OCUP
OCX
ODC
ODFL
ODP
ODT
OEC
OEG
OESX
OFED
OFG
OFIX
OFLX
OFS
OGE
OGEN
OGI
OGS
OI
OIA
OII
OIIM
OIS
OJSCY
OKE
OKTA
OLB
OLED
OLLI
OLMA
OLN
OLP
OM
OMAB
OMC
OMCL
OMER
OMEX
OMF
OMI
OMP
end time: 2020-12-26 00:39:00.039058
i=3400
ON
ONB
ONCR
ONCS
ONCT
ONCY
ONE
ONEM
ONEW
ONTO
ONTX
ONVO
OOMA
OPBK
OPCH
OPES
OPGN
OPHC
OPI
OPK
OPNT
OPOF
OPRA
OPRT
OPRX
OPTN
OPTT
OPY
OR
ORA
ORAN
ORBC
ORC
ORCC
ORCL
ORGO
ORGS
ORI
ORIC
ORLY
ORMP
ORN
ORRF
ORSN
ORTX
OSB
OSBC
OSG
OSIS
OSK
OSMT
OSN
OSPN
OSS
OSTK
OSUR
OSW
OTEL
OTEX
OTIC
OTIS
OTLK
OTRK
OTTR
OTTW
OVBC
OVID
OVLY
OVV
OXBR
OXFD
OXLC
OXM
OXSQ
OXY
OYST
OZK
OZON
PAA
PAAS
PAC
PACB
PACD
PACK
PACW
PAE
PAG
PAGP
PAGS
PAHC
PAIC
PAM
PANA
PAND
PANL
PANW
PAR
PARR
PASG
PATI
end time: 2020-12-26 00:39:30.022134
i=3500
PATK
PAVM
PAYA
PAYC
PAYS
PAYX
PB
PBA
PBCT
PBF
PBFS
PBFX
PBH
PBHC
PBI
PBIP
PBLA
PBPB
PBR
PBT
PBTS
PBYI
PCAR
PCB
PCG
PCH
PCOM
PCPL
PCRX
PCSA
PCSB
PCTI
PCTY
PCVX
PCYG
PCYO
PD
PDAC
PDCE
PDCO
PDD
PDEX
PDFS
PDLB
PDLI
PDM
PDS
PDS

WNC
WNEB
WNS
WOR
WORK
WORX
WOW
WPF
WPM
WPP
WPRT
WPX
WRAP
WRB
WRE
WRK
WRLD
WRN
WRTC
end time: 2020-12-26 00:47:28.182600
i=5000
WSBC
WSBF
WSC
WSFS
WSG
WSM
WSO
WSR
WST
WSTG
WSTL
WTBA
WTER
WTFC
WTI
WTM
WTRE
WTRG
WTRH
WTS
WTT
WTTR
WU
WVE
WVFC
WVVI
WW
WWD
WWE
WWR
WWW
WY
WYND
WYNN
WYY
X
XAIR
XBIO
XBIT
XCUR
XEC
XEL
XELA
XELB
XENE
XENT
XERS
XFOR
XGN
XHR
XLB
XLC
XLE
XLF
XLI
XLK
XLNX
XLP
XLRE
XLRN
XLU
XLV
XLY
XNCR
XNET
XOG
XOM
XOMA
XONE
XP
XPEL
XPER
XPEV
XPL
XPO
XRAY
XRX
XSPA
XTLB
XTNT
XXII
XYF
XYL
Y
YAC
YALA
YAYO
YCBD
YELP
YETI
YEXT
YGMZ
YGYI
YI
YJ
YMAB
YNDX
YORW
YPF
YRCW
end time: 2020-12-26 00:47:59.616126
i=5100
YRD
YSG
YTEN
YTRA
YUM
YUMC
YVR
YY
Z
ZAGG
ZBH
ZBRA
ZCMD
ZDGE
ZEAL
ZEN
ZEUS
ZG
ZGNX
ZGYH
ZI
ZION
ZIOP
ZIXI
ZKIN
ZLAB
ZM
ZN
ZNGA
ZNH
ZNTL
ZOM
ZS
ZSAN
ZTO
ZTS
ZUMZ
ZUO
ZVO
ZYME
ZYNE
ZYXI
end time: 2020-12-26 00:48:13.760763
